# Structured Data Demo



In [2]:
# Ish via https://til.simonwillison.net/python/sqlite-in-pyodide
from js import fetch
 
async def load_file_into_in_mem_filesystem(url, fn=None):
    """Load a file from a URL into an in-memory filesystem."""
     
    # Create a filename if required
    fn = fn if fn is not None else url.split("/")[-1]
     
    # Fetch file from URL
    res = await fetch(url)
     
    # Buffer it
    buffer = await res.arrayBuffer()
     
    # Write file to in-memory file system
    open(fn, "wb").write(bytes(buffer.valueOf().to_py()))
 
    return fn


In [3]:
url="https://raw.githubusercontent.com/UTCSheffield/OCR-Unit-7-Data-analysis-and-design/main/content/data/chinook.db"
 
db_file = await load_file_into_in_mem_filesystem(url)

In [4]:
import sqlite3
import pandas as pd

# Open database connection
c = sqlite3.connect(db_file)
 
employees = pd.read_sql_query("SELECT * FROM employees", c)

# Verify that result of SQL query is stored in the dataframe
employees


,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
5,6,Mitchell,Michael,IT Manager,1.0,1973-07-01 00:00:00,2003-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
6,7,King,Robert,IT Staff,6.0,1970-05-29 00:00:00,2004-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
7,8,Callahan,Laura,IT Staff,6.0,1968-01-09 00:00:00,2004-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [5]:

employees_fitered = pd.read_sql_query("SELECT EmployeeId, FirstName, LastName, Email, Title FROM employees WHERE Title='IT Staff';", c)


employees_fitered

,EmployeeId,FirstName,LastName,Email,Title
0,7,Robert,King,robert@chinookcorp.com,IT Staff
1,8,Laura,Callahan,laura@chinookcorp.com,IT Staff


In [10]:
albums = pd.read_sql_query("SELECT * FROM albums JOIN artists ON (albums.ArtistId = artists.ArtistId);", c)
albums

,AlbumId,Title,ArtistId,ArtistId,Name
0,1,For Those About To Rock We Salute You,1,1,AC/DC
1,2,Balls to the Wall,2,2,Accept
2,3,Restless and Wild,2,2,Accept
3,4,Let There Be Rock,1,1,AC/DC
4,5,Big Ones,3,3,Aerosmith
...,...,...,...,...,...
342,343,Respighi:Pines of Rome,226,226,Eugene Ormandy
343,344,Schubert: The Late String Quartets & String Qu...,272,272,Emerson String Quartet
344,345,Monteverdi: L'Orfeo,273,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
345,346,Mozart: Chamber Music,274,274,Nash Ensemble


In [11]:
tracks = pd.read_sql_query("SELECT * FROM tracks JOIN albums ON (tracks.AlbumId = albums.AlbumId) JOIN artists ON (albums.ArtistId = artists.ArtistId);", c)
tracks

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice,AlbumId,Title,ArtistId,ArtistId,Name
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,1,For Those About To Rock We Salute You,1,1,AC/DC
1,6,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,0.99,1,For Those About To Rock We Salute You,1,1,AC/DC
2,7,Let's Get It Up,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",233926,7636561,0.99,1,For Those About To Rock We Salute You,1,1,AC/DC
3,8,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,0.99,1,For Those About To Rock We Salute You,1,1,AC/DC
4,9,Snowballed,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",203102,6599424,0.99,1,For Those About To Rock We Salute You,1,1,AC/DC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3498,3499,Pini Di Roma (Pinien Von Rom) \ I Pini Della V...,343,2,24,None,286741,4718950,0.99,343,Respighi:Pines of Rome,226,226,Eugene Ormandy
3499,3500,"String Quartet No. 12 in C Minor, D. 703 ""Quar...",344,2,24,Franz Schubert,139200,2283131,0.99,344,Schubert: The Late String Quartets & String Qu...,272,272,Emerson String Quartet
3500,3501,"L'orfeo, Act 3, Sinfonia (Orchestra)",345,2,24,Claudio Monteverdi,66639,1189062,0.99,345,Monteverdi: L'Orfeo,273,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
3501,3502,"Quintet for Horn, Violin, 2 Violas, and Cello ...",346,2,24,Wolfgang Amadeus Mozart,221331,3665114,0.99,346,Mozart: Chamber Music,274,274,Nash Ensemble
